<a href="https://colab.research.google.com/github/dsgustavomoreira/Tranformers-Classificacao-de-Revisoes-e-Aplicacoes-Diversas/blob/main/Transformers_Aplica%C3%A7%C3%B5es_Diversas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1 - Problema de Negócio** - Classificar as revisões dos produtos no ecommerce Olist

# **2 - Instalação das Bibliotecas**

In [ ]:
!pip install transformers datasets
!pip install kaggle
!pip install accelerate -um
!pip install transformers[torch]
!pip install gradio

In [2]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d olistbr/brazilian-ecommerce
!unzip brazilian-ecommerce.zip

 96% 41.0M/42.6M [00:00<00:00, 90.2MB/s]
100% 42.6M/42.6M [00:00<00:00, 87.0MB/s]
Archive:  brazilian-ecommerce.zip
  inflating: olist_customers_dataset.csv  
  inflating: olist_geolocation_dataset.csv  
  inflating: olist_order_items_dataset.csv  
  inflating: olist_order_payments_dataset.csv  
  inflating: olist_order_reviews_dataset.csv  
  inflating: olist_orders_dataset.csv  
  inflating: olist_products_dataset.csv  
  inflating: olist_sellers_dataset.csv  
  inflating: product_category_name_translation.csv  


# **3 - Importação das Bibliotecas**

In [33]:
import pandas as pd
import numpy as np
import gradio as gr
import tensorflow as tf
from transformers import AutoTokenizer, BertTokenizer, BertTokenizerFast, TFAutoModelForSequenceClassification, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# **4 - Análise e Processamento dos Dados**

In [4]:
file_path = '/content/olist_order_reviews_dataset.csv'
df = pd.read_csv(file_path)

In [5]:
df = df.dropna(subset=['review_comment_message', 'review_score'])

# Criação de uma coluna de sentimento onde review_score >= 4 são positivos  (label 1) e o restante é negativo (label 0)
df.loc[:, 'sentiment'] = df['review_score'].apply(lambda score: 1 if score >= 4 else 0)

<ipython-input-5-6df6ae11bab3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'sentiment'] = df['review_score'].apply(lambda score: 1 if score >= 4 else 0)


In [6]:
df.review_score.head(10)

3     5
4     5
9     4
12    4
15    5
16    2
19    1
22    5
24    5
27    5
Name: review_score, dtype: int64

In [7]:
df.sentiment.head(10)

3     1
4     1
9     1
12    1
15    1
16    0
19    0
22    1
24    1
27    1
Name: sentiment, dtype: int64

# **5 - Máquina Preditiva** - Transformers

In [8]:
train_texts, test_texts, train_labels, test_labels = train_test_split(df['review_comment_message'], df['sentiment'], test_size=0.1)

# Carregar o tokenizer
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

# Tokenização e preparação dos inputs
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
# Carregar o modelo
model = TFAutoModelForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=2)

# Compilar o modelo
optimizer = 'adam'
loss = SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Converter as codificações em tf.data.Dataset para alimentar o modelo
train_dataset = (
    tf.data.Dataset.from_tensor_slices((
        dict(train_encodings),
        train_labels
    )).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
)
test_dataset = (
    tf.data.Dataset.from_tensor_slices((
        dict(test_encodings),
        test_labels
    )).batch(32).prefetch(tf.data.AUTOTUNE)
)

model.fit(train_dataset, validation_data=test_dataset, epochs=1)

tf_model.h5:   0%|          | 0.00/529M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier', 'bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1153/1153 [==============================] - 1045s 832ms/step - loss: 0.6586 - accuracy: 0.6446 - val_loss: 0.6715 - val_accuracy: 0.6508


# **6 - Salvar e Compactar o Modelo pré-treinado**

In [25]:
# Salvar o modelo
model.save_pretrained('/content/dl_sentiment_model')
tokenizer.save_pretrained('/content/dl_to_sentiment_model')

('/content/dl_to_sentiment_model/tokenizer_config.json',
 '/content/dl_to_sentiment_model/special_tokens_map.json',
 '/content/dl_to_sentiment_model/vocab.txt',
 '/content/dl_to_sentiment_model/added_tokens.json',
 '/content/dl_to_sentiment_model/tokenizer.json')

In [16]:
!zip -r /content/dl_sentiment_model.zip /content/dl_sentiment_model

# Código para baixar o arquivo zip via Colab
from google.colab import files
files.download('/content/dl_sentiment_model.zip')

  adding: content/dl_sentiment_model/ (stored 0%)
  adding: content/dl_sentiment_model/tf_model.h5 (deflated 7%)
  adding: content/dl_sentiment_model/.zip (stored 0%)
  adding: content/dl_sentiment_model/config.json (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **7 - Utilizar o Modelo**

In [31]:
# Carregar o tokenizer e o modelo treinados
tokenizer = BertTokenizer.from_pretrained('/content/dl_sentiment_model')
modelo = TFBertForSequenceClassification.from_pretrained('/content/dl_sentiment_model')

# Função para preparar as reviews
def prepare_reviews(reviews, tokenizer):
    max_length = 512
    return tokenizer(reviews, padding=True, truncation=True, max_length=max_length, return_tensors='tf')

# Função para prever o sentimento das reviews
def predict_sentiment(reviews, tokenizer, model):
    # Preparar as reviews
    encoded_input = prepare_reviews(reviews, tokenizer)
    # Prever
    outputs = model(encoded_input)
    # Transformar logits para probabilidades
    probabilities = tf.nn.softmax(outputs.logits, axis=-1)
    # Buscar o índice do valor máximo como a classe predita
    predictions = tf.argmax(probabilities, axis=1)
    return predictions.numpy()

# Novas reviews para análise
new_reviews = [
    "Produto de muito boa qualidade, prazo foi cumprido corretamente e o produto chegou intacto",
    "Péssimo! Não é adequado para o uso, apresentou defeito de fábrica, não considero comprar novamente!"
]

# Prever o sentimento das novas reviews
sentiments = predict_sentiment(new_reviews, tokenizer, model)
for review, sentiment in zip(new_reviews, sentiments):
    print(f"Review: {review}\nSentiment: {'Positive' if sentiment == 1 else 'Negative'}\n")

Some layers from the model checkpoint at /content/dl_sentiment_model were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/dl_sentiment_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Review: Produto de muito boa qualidade, prazo foi cumprido corretamente e o produto chegou intacto
Sentiment: Positive

Review: Péssimo! Não é adequado para o uso, apresentou defeito de fábrica, não considero comprar novamente!
Sentiment: Positive



In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('/content/dl_sentiment_model')
model = TFBertForSequenceClassification.from_pretrained('/content/dl_sentiment_model')

# Função para fazer a inferência
def classify_review(review):
    inputs = tokenizer(review, return_tensors="tf", padding=True, truncation=True)
    outputs = model(inputs)
    predictions = tf.nn.softmax(outputs.logits, axis=-1)
    sentiment = tf.argmax(predictions, axis=-1).numpy()
    return 'Positivo' if sentiment == 1 else 'Negativo'

# Criar a interface de Gradio
iface = gr.Interface(fn=classify_review,
                     inputs=gr.Textbox(lines=2, placeholder="Digite aqui a sua review..."),
                     outputs="text",
                     title="Classificador de Sentimento de Reviews",
                     description="Modelo para classificar o sentimento de reviews como Positivo ou Negativo.")

# Lançar a aplicação com debug=True para mostrar erros completos se eles ocorrerem.
iface.launch(debug=True)

Some layers from the model checkpoint at /content/dl_sentiment_model were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/dl_sentiment_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a59a324531cd7846a8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


# **Aplicações com Hugging Face**

In [ ]:
!pip install transformers --upgrade
!pip install --upgrade huggingface_hub

In [ ]:
from huggingface_hub import HfApi

api = HfApi()

try:
    models = api.list_models()

    models_list = list(models)

    print(f'Total de modelos encontrados: {len(models_list)}')

    if models_list:
        for model in models_list[0:10]:
            print(model.modelId)
    else:
        print("Nenhum modelo foi encontrado.")
except Exception as e:
    print(f'Ocorreu um erro: {e}')

Total de modelos encontrados: 566044
albert/albert-base-v1
albert/albert-base-v2
albert/albert-large-v1
albert/albert-large-v2
albert/albert-xlarge-v1
albert/albert-xlarge-v2
albert/albert-xxlarge-v1
albert/albert-xxlarge-v2
google-bert/bert-base-cased-finetuned-mrpc
google-bert/bert-base-cased


Listagem por Tipo de Modelo (Task)

In [ ]:
from transformers import pipeline
st = f'I do not like horror movies'
seq = pipeline(task='text-classification', model='nlptown/bert-base-multilingual-uncased-sentiment')
print(f'Result: {seq(st)}')

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Result: [{'label': '2 stars', 'score': 0.4139205515384674}]


In [ ]:
from transformers import pipeline

sentence = r"""


Alice começava a ficar muito cansada de sentar-se ao lado da irmã na margem, e de não ter nada para fazer:
uma ou duas vezes ela tinha espiado o livro que a irmã estava lendo, mas ele não tinha imagens nem diálogos.
"E para que serve um livro", pensou Alice, "sem imagens nem conversas?"
Então ela estava considerando em sua própria mente
(da melhor forma que podia, pois o dia quente a fazia sentir-se muito sonolenta e abobada),
 se o prazer de fazer uma corrente de margaridas valeria o esforço de levantar-se e colher as margaridas,
 quando de repente um Coelho Branco com olhos cor-de-rosa passou correndo perto dela


"""
output = pipeline('question-answering', model='csarron/roberta-base-squad-v1')
question = output(question='Quem está lendo o livro?', context=sentence)
print(f'Resposta {question["answer"]}')

Some weights of the model checkpoint at csarron/roberta-base-squad-v1 were not used when initializing RobertaForQuestionAnswering: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Resposta a irmã


In [ ]:
from transformers import pipeline
txt = r'''
Atualmente, o aprendizado de máquina (Machine Learning) é um tópico importante. Todo mundo fala sobre essa nova ferramenta dos Cientistas de Dados, os modelos são implementados em vários domínios, sejam eles na saúde, finanças, segurança, varejo...cada vez mais startups nascem com o DNA do Machine Learning.
Ao mesmo tempo, o aprendizado de máquina é um campo complexo, com várias dimensões diferentes. Às vezes, mesmo especialistas técnicos experientes, dificilmente conseguem imaginar todo o universo do Machine Learning e seu lugar neste universo. Muitas pessoas estão apenas curiosas sobre ML e não estão profundamente imersas no assunto.
Para esses, também é importante entender a estrutura do aprendizado de máquina. O aprendizado de máquina é um subcampo da Ciência de Dados, que é a área mais ampla. Pensando nisso, trago para você alguns entendimentos sobre os tipos de tarefas do processo do aprendizado de máquina para que você possa se familiarizar com essas terminologias, que nada mais são do que formas de classificar as tarefas do ML.
'''
nlp = pipeline(task='summarization')
nlp(txt, max_length=130, min_length=30)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'summary_text': ' O aprendizado de máquina (Machine Learning) is a subcampo da Ciência de Dados . Muitas pessoas estão apenas curiosas sobre ML e não estão profundamentamente imersas no assunto .'}]